# Partie 4.2 - Visualisations avancees avec Seaborn

Ce notebook produit 6 visualisations avancees a l'aide de la bibliotheque Seaborn,
a partir des donnees enrichies de consommation energetique des batiments publics.

**Objectifs :**
- Pairplot des consommations par saison
- Violin plot de la consommation electrique par type de batiment
- Heatmap annotee de la matrice de correlation
- FacetGrid de l'evolution mensuelle par commune
- Jointplot surface vs consommation
- Catplot par classe energetique et type de batiment

**Sortie :** 6 figures PNG (300 dpi) dans `../output/figures/`

In [ ]:
# --- Imports et configuration ---

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import warnings

warnings.filterwarnings("ignore")

# Style Seaborn
sns.set_theme(style="whitegrid", palette="husl")

# Configuration matplotlib pour les labels en francais
plt.rcParams.update({
    "figure.figsize": (12, 8),
    "font.size": 12,
    "axes.titlesize": 14,
    "axes.labelsize": 12
})

# Creation du repertoire de sortie
os.makedirs("../output/figures", exist_ok=True)

# --- Chargement des donnees enrichies ---
df = pd.read_parquet("../output/consommations_enrichies.parquet")

print(f"Dimensions du jeu de donnees : {df.shape}")
print(f"Colonnes : {list(df.columns)}")
print(f"\nTypes de batiment : {df['type'].unique()}")
print(f"Types d'energie   : {df['type_energie'].unique()}")
print(f"Saisons           : {df['saison'].unique()}")
df.head()

## Figure 1 : Pairplot des consommations par saison

Ce pairplot presente les relations croisees entre les consommations d'electricite,
de gaz et d'eau, colorees par saison. On pivote les donnees pour obtenir une colonne
par type d'energie.

In [ ]:
# --- Figure 1 : Pairplot des consommations par saison ---

# Pivot : une colonne par type d'energie, agrégé par batiment, mois et saison
df_pivot = (
    df.groupby(["batiment_id", "mois", "saison", "type_energie"])["consommation"]
    .sum()
    .reset_index()
    .pivot_table(
        index=["batiment_id", "mois", "saison"],
        columns="type_energie",
        values="consommation",
        aggfunc="sum"
    )
    .reset_index()
)

# Renommage des colonnes pour le graphique
rename_energie = {
    "electricite": "Electricite (kWh)",
    "gaz": "Gaz (kWh)",
    "eau": "Eau (m3)"
}
cols_energie = [c for c in ["electricite", "gaz", "eau"] if c in df_pivot.columns]
df_pivot = df_pivot.rename(columns=rename_energie)
cols_plot = [rename_energie[c] for c in cols_energie]

# Suppression des valeurs manquantes
df_pivot = df_pivot.dropna(subset=cols_plot)

# Echantillonnage pour la performance si necessaire
if len(df_pivot) > 2000:
    df_pivot_sample = df_pivot.sample(n=2000, random_state=42)
else:
    df_pivot_sample = df_pivot.copy()

# Ordre des saisons
ordre_saisons = ["hiver", "printemps", "ete", "automne"]

g = sns.pairplot(
    df_pivot_sample,
    vars=cols_plot,
    hue="saison",
    hue_order=ordre_saisons,
    diag_kind="kde",
    plot_kws={"alpha": 0.6, "s": 30},
    palette="husl",
    corner=False
)

g.figure.suptitle(
    "Relations entre les consommations d'energie par saison",
    y=1.02, fontsize=16, fontweight="bold"
)

g.savefig(
    "../output/figures/fig06_pairplot_saisons.png",
    dpi=300, bbox_inches="tight"
)
plt.show()
print("Figure 1 sauvegardee : fig06_pairplot_saisons.png")

## Figure 2 : Violin plot de la consommation electrique par type de batiment

Le violin plot permet de visualiser la distribution complete de la consommation
electrique pour chaque type de batiment, avec les quartiles.

In [ ]:
# --- Figure 2 : Violin plot - consommation electrique par type de batiment ---

df_elec = df[df["type_energie"] == "electricite"].copy()

# Ordre des types de batiment par consommation mediane decroissante
ordre_types = (
    df_elec.groupby("type")["consommation"]
    .median()
    .sort_values(ascending=False)
    .index.tolist()
)

fig, ax = plt.subplots(figsize=(14, 7))

sns.violinplot(
    data=df_elec,
    x="type",
    y="consommation",
    order=ordre_types,
    inner="quartile",
    palette="husl",
    linewidth=1.2,
    saturation=0.85,
    ax=ax
)

ax.set_title(
    "Distribution de la consommation electrique par type de batiment",
    fontsize=16, fontweight="bold", pad=15
)
ax.set_xlabel("Type de batiment", fontsize=13)
ax.set_ylabel("Consommation electrique (kWh)", fontsize=13)
ax.tick_params(axis="x", labelsize=11)
ax.tick_params(axis="y", labelsize=11)

# Annotation : mediane par type
for i, t in enumerate(ordre_types):
    mediane = df_elec[df_elec["type"] == t]["consommation"].median()
    ax.text(
        i, mediane, f"{mediane:.0f}",
        ha="center", va="bottom", fontsize=9,
        fontweight="bold", color="black"
    )

plt.tight_layout()
fig.savefig(
    "../output/figures/fig07_violin_electricite.png",
    dpi=300, bbox_inches="tight"
)
plt.show()
print("Figure 2 sauvegardee : fig07_violin_electricite.png")

## Figure 3 : Heatmap annotee de la matrice de correlation

La matrice de correlation met en evidence les relations lineaires entre les
principales variables numeriques du jeu de donnees. Le triangle superieur est
masque pour eviter la redondance.

In [ ]:
# --- Figure 3 : Heatmap de la matrice de correlation ---

# Selection des variables numeriques cles
colonnes_numeriques = [
    "consommation", "surface_m2", "nb_occupants_moyen",
    "temperature_c", "humidite_pct", "cout",
    "consommation_par_m2", "consommation_par_occupant"
]

# Filtrer les colonnes presentes dans le DataFrame
colonnes_presentes = [c for c in colonnes_numeriques if c in df.columns]

# Labels en francais pour l'affichage
labels_fr = {
    "consommation": "Consommation",
    "surface_m2": "Surface (m2)",
    "nb_occupants_moyen": "Nb occupants",
    "temperature_c": "Temperature (C)",
    "humidite_pct": "Humidite (%)",
    "cout": "Cout (EUR)",
    "consommation_par_m2": "Conso/m2",
    "consommation_par_occupant": "Conso/occupant"
}

# Calcul de la matrice de correlation
matrice_corr = df[colonnes_presentes].corr()

# Renommage des index et colonnes
matrice_corr.index = [labels_fr.get(c, c) for c in matrice_corr.index]
matrice_corr.columns = [labels_fr.get(c, c) for c in matrice_corr.columns]

# Masque du triangle superieur
masque = np.triu(np.ones_like(matrice_corr, dtype=bool))

fig, ax = plt.subplots(figsize=(12, 10))

sns.heatmap(
    matrice_corr,
    mask=masque,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    center=0,
    vmin=-1, vmax=1,
    square=True,
    linewidths=0.5,
    linecolor="white",
    cbar_kws={"label": "Coefficient de correlation", "shrink": 0.8},
    annot_kws={"size": 10},
    ax=ax
)

ax.set_title(
    "Matrice de correlation des variables numeriques",
    fontsize=16, fontweight="bold", pad=20
)
ax.tick_params(axis="x", rotation=45, labelsize=11)
ax.tick_params(axis="y", rotation=0, labelsize=11)

plt.tight_layout()
fig.savefig(
    "../output/figures/fig08_heatmap_correlation.png",
    dpi=300, bbox_inches="tight"
)
plt.show()
print("Figure 3 sauvegardee : fig08_heatmap_correlation.png")

## Figure 4 : FacetGrid - Evolution mensuelle par commune (top 6)

Ce FacetGrid presente l'evolution mensuelle de la consommation energetique
pour les 6 communes les plus consommatrices, chacune dans un sous-graphique distinct.

In [ ]:
# --- Figure 4 : FacetGrid - Evolution mensuelle par commune (top 6) ---

# Identification des 6 communes les plus consommatrices
top6_communes = (
    df.groupby("commune")["consommation"]
    .sum()
    .nlargest(6)
    .index.tolist()
)

print(f"Top 6 communes par consommation totale : {top6_communes}")

# Filtrage et agregation mensuelle
df_top6 = df[df["commune"].isin(top6_communes)].copy()

df_mensuel = (
    df_top6
    .groupby(["commune", "mois"])["consommation"]
    .sum()
    .reset_index()
    .sort_values(["commune", "mois"])
)

# FacetGrid
g = sns.FacetGrid(
    df_mensuel,
    col="commune",
    col_wrap=3,
    col_order=top6_communes,
    height=4,
    aspect=1.3,
    sharey=False
)

g.map_dataframe(
    sns.lineplot,
    x="mois",
    y="consommation",
    marker="o",
    color="steelblue",
    linewidth=1.5,
    markersize=5
)

g.set_titles(col_template="{col_name}", size=13, fontweight="bold")
g.set_axis_labels("Mois", "Consommation totale", fontsize=11)

# Rotation des labels de l'axe X
for ax in g.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)
        label.set_fontsize(8)
    ax.yaxis.set_major_formatter(
        plt.FuncFormatter(lambda x, _: f"{x:,.0f}")
    )

g.figure.suptitle(
    "Evolution mensuelle de la consommation par commune (top 6)",
    y=1.03, fontsize=16, fontweight="bold"
)

g.tight_layout()
g.savefig(
    "../output/figures/fig09_facetgrid_communes.png",
    dpi=300, bbox_inches="tight"
)
plt.show()
print("Figure 4 sauvegardee : fig09_facetgrid_communes.png")

## Figure 5 : Jointplot - Surface vs consommation

Ce jointplot met en evidence la relation entre la surface des batiments et
leur consommation totale, avec les distributions marginales.

In [ ]:
# --- Figure 5 : Jointplot - Surface vs consommation ---

# Agregation : consommation totale par batiment
df_batiment = (
    df.groupby(["batiment_id", "nom", "type", "surface_m2"])
    .agg(consommation_totale=("consommation", "sum"))
    .reset_index()
)

g = sns.jointplot(
    data=df_batiment,
    x="surface_m2",
    y="consommation_totale",
    hue="type",
    kind="scatter",
    height=10,
    ratio=4,
    marginal_kws={"common_norm": False},
    joint_kws={"alpha": 0.7, "s": 60, "edgecolor": "white", "linewidth": 0.5},
    palette="husl"
)

g.set_axis_labels(
    "Surface du batiment (m2)",
    "Consommation totale",
    fontsize=13
)

g.figure.suptitle(
    "Relation entre surface et consommation totale par batiment",
    y=1.02, fontsize=16, fontweight="bold"
)

g.savefig(
    "../output/figures/fig10_jointplot_surface.png",
    dpi=300, bbox_inches="tight"
)
plt.show()
print("Figure 5 sauvegardee : fig10_jointplot_surface.png")

## Figure 6 : Catplot - Consommation par classe energetique et type de batiment

Ce catplot compare la consommation par m2 selon la classe energetique (DPE)
et le type de batiment, mettant en evidence l'impact de la performance
energetique sur la consommation reelle.

In [ ]:
# --- Figure 6 : Catplot - Consommation par classe energetique et type ---

# Ordre des classes energetiques de A a G
ordre_classes = ["A", "B", "C", "D", "E", "F", "G"]
classes_presentes = [c for c in ordre_classes if c in df["classe_energetique"].unique()]

g = sns.catplot(
    data=df[df["classe_energetique"].isin(classes_presentes)],
    x="classe_energetique",
    y="consommation_par_m2",
    hue="type",
    kind="bar",
    order=classes_presentes,
    palette="husl",
    height=7,
    aspect=1.6,
    ci=95,
    errwidth=1.2,
    capsize=0.05,
    edgecolor="white",
    linewidth=0.8
)

g.set_axis_labels(
    "Classe energetique (DPE)",
    "Consommation par m2",
    fontsize=13
)

g.figure.suptitle(
    "Consommation par m2 selon la classe energetique et le type de batiment",
    y=1.02, fontsize=16, fontweight="bold"
)

g.legend.set_title("Type de batiment")
g._legend.set_bbox_to_anchor((1.02, 0.5))

g.tight_layout()
g.savefig(
    "../output/figures/fig11_catplot_dpe_type.png",
    dpi=300, bbox_inches="tight"
)
plt.show()
print("Figure 6 sauvegardee : fig11_catplot_dpe_type.png")

## Conclusion

Ce notebook a produit 6 visualisations avancees avec Seaborn :

1. **Pairplot par saison** : met en evidence les correlations croisees entre electricite, gaz et eau,
   avec des profils de consommation distincts selon les saisons (consommation de gaz plus elevee en hiver).

2. **Violin plot electrique** : revele la distribution complete de la consommation electrique par type
   de batiment, avec les quartiles. Les piscines et gymnases presentent generalement les distributions
   les plus larges.

3. **Heatmap de correlation** : identifie les correlations significatives entre les variables numeriques.
   La consommation est fortement correlee au cout et a la surface.

4. **FacetGrid mensuel** : compare l'evolution temporelle de la consommation pour les 6 communes
   les plus consommatrices, revelant des profils saisonniers distincts.

5. **Jointplot surface/consommation** : confirme la relation positive entre surface et consommation
   totale, avec des distributions marginales par type de batiment.

6. **Catplot DPE/type** : demontre l'impact de la classe energetique sur la consommation par m2,
   avec une progression nette de A (faible) a G (elevee), modulee par le type de batiment.

Les 6 figures ont ete sauvegardees au format PNG (300 dpi) dans le repertoire `../output/figures/`.